In [1]:
import re
import json
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import numpy as np
import pandas as pd 
from pydantic import BaseModel
from openai import OpenAI
from typing import List

In [2]:
df = pd.read_excel('../Data/Places Dataset.xlsx')

In [5]:
client = OpenAI()

class Activity(BaseModel):
    name: str

class Place(BaseModel):
    activities: List[Activity]
    Geographical: str
    Historical: str
    Religious: str
    Natural: str
    Entertainment: str
    Accommodation: str
    Shopping: str
    Food: str


In [6]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "system", "content": "You are are experieced in Sri Lankan tourism. you should able to give knowledge about the places in Sri Lanka"},
        {"role": "user", "content": "Kandy"}
    ],
    response_format=Place
  )

In [12]:
activities = completion.choices[0].message.parsed.activities

# To print the activity names
for activity in activities:
    print(activity.name)

Cultural Dance Performance
Temple of the Tooth Relic Tour
Botanical Gardens Visit
City Tour
Shopping at Local Markets


In [28]:
def get_info(location):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are experienced in Sri Lankan tourism. You should be able to give knowledge about the places in Sri Lanka without promoting hotels or restaurants."},
            {"role": "user", "content": f"{location}"}
        ],
        response_format=Place
    )
    
    return completion.choices[0].message.parsed


In [32]:
def get_acts(place_info):
    activities = [activity.name for activity in place_info.activities]
    return activities


def get_geo(place_info):
    geo = place_info.Geographical
    return geo


def get_hsitoric(place_info):
    history = place_info.Historical
    return history


def get_religious(place_info):
    religion = place_info.Religious
    return religion

    
def get_natural(place_info):
    nature = place_info.Natural
    return nature


def get_entertainment(place_info):
    entertainment = place_info.Entertainment
    return entertainment


def get_accommodation(place_info):
    accommodation = place_info.Accommodation
    return accommodation


def get_shopping(place_info):
    shopping = place_info.Shopping
    return shopping


def get_food(place_info):
    food = place_info.Food
    return food

In [30]:
df["place_info"] = df["name"].apply(get_info)

In [33]:
df["activities"] = df["place_info"].apply(get_acts)
df["Geographical"] = df["place_info"].apply(get_geo)
df["Historical"] = df["place_info"].apply(get_hsitoric)
df["Religious"] = df["place_info"].apply(get_religious)
df["Natural"] = df["place_info"].apply(get_natural)
df["Entertainment"] = df["place_info"].apply(get_entertainment)
df["Accommodation"] = df["place_info"].apply(get_accommodation)
df["Shopping"] = df["place_info"].apply(get_shopping)
df["Food"] = df["place_info"].apply(get_food)

In [34]:
# exoport the data to a csv file

df.to_csv("../Data/Places Dataset Enriched Classified Reviews Filled.csv", index=False)


In [36]:
df.drop(columns=["place_info"], inplace=True)

In [38]:
df.to_csv("../Data/Places Dataset Enriched.csv", index=False)